# Environement Setup

In [1]:
import sys, os
try:
    from google.colab import drive, userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

REPO_NAME = 'MistakeDetection'

if IS_COLAB:
    print("☁️ Colab rilevato.")
    if not os.path.exists('/content/drive'): drive.mount('/content/drive')

    GITHUB_USER = 'MarcoPernoVDP'
    try:
        TOKEN = userdata.get('GITHUB_TOKEN')
        REPO_URL = f'https://{TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git'
    except:
        REPO_URL = f'https://github.com/{GITHUB_USER}/{REPO_NAME}.git'

    ROOT_DIR = f'/content/{REPO_NAME}'
    if not os.path.exists(ROOT_DIR):
        !git clone {REPO_URL}
    else:
        %cd {ROOT_DIR}
        !git pull
        %cd /content


else:
    print("Ambiente locale rilevato.")
    ROOT_DIR = os.getcwd()
    while not os.path.exists(os.path.join(ROOT_DIR, '.gitignore')) and ROOT_DIR != os.path.dirname(ROOT_DIR):
        ROOT_DIR = os.path.dirname(ROOT_DIR)

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)


Ambiente locale rilevato.


# Dataset Setup

In [2]:
from utils import setup_project
# Ora puoi passare agli import del modello
from dataset.capitain_cook_4d_mlp_dataset import CaptainCook4DMLP_Dataset, DatasetSource
from models.BaselineV3_LSTM import BaselineV3_LSTM
from dataset.utils import SplitType

# Esegue: Setup Dati (unzip/copy), Login WandB, Setup Device
device = setup_project.initialize(ROOT_DIR)

# Import wandb
import wandb

Setup Progetto in: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection
source_path: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\_file
Setup Dati da: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\_file
Inizio setup dati...
   Sorgente: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\_file
   Destinazione: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\data
Copia cartella: annotation_json...
Copia cartella: omnivore...
✅ Setup completato! Dati pronti in: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\data


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\marco\_netrc
wandb: Currently logged in as: s339450 (s339450-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


WandB Logged in.
Device: cuda


# Configuration

In [3]:
# Configurazione esperimento
DATASET_SOURCE = DatasetSource.OMNIVORE
SPLIT_TYPE = SplitType.STEP_ID

config = {
    "architecture": "BaselineV3_LSTM_" + DATASET_SOURCE.value + "_" + SPLIT_TYPE.value,
    "dataset": "CaptainCook4D",
    "feature_extractor": DATASET_SOURCE.value,
    "input_dim": DATASET_SOURCE.input_dims(),
    "batch_size": 1,  # DEVE essere 1 per sequenze di lunghezza variabile
    "learning_rate": 1e-5,
    "epochs": 10,
    "pos_weight": 1.5,
    "optimizer": "Adam",
    "loss_function": "BCEWithLogitsLoss",
    "seed": 42,
    "split_type": SPLIT_TYPE.value
}

# Dataset Split

In [4]:
import os
from dataset.capitain_cook_4d_mlp_dataset import DatasetSource
from dataset.capitain_cook_4d_transformer_dataset import CaptainCook4DTransformer_Dataset
from dataset.utils import get_transformer_loaders

try:
    full_dataset = CaptainCook4DTransformer_Dataset(
        dataset_source=DATASET_SOURCE,
        root_dir=ROOT_DIR
    )
    train_loader, val_loader, test_loader = get_transformer_loaders(
        full_dataset,
        batch_size=config["batch_size"],
        seed=config["seed"],
        split_type=SPLIT_TYPE
    )

except Exception as e:
    print(f"❌ Errore: {e}")

Loading from: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\data\omnivore...
Dataset creato: 200 step completi da 8828 secondi

DATASET INFO [TRANSFORMER - STEP-BASED]
   Total Steps: 200
   Features per second: 1024
   Step duration: min=3s, max=184s, avg=44.14s
FULL DATASET       | Tot: 200    | OK: 117   (58.5%) | ERR: 83    (41.5%) | Ratio: 1:1.4
-------------------------------------------------------------------------------------
TRAIN SET          | Tot: 140    | OK: 83    (59.3%) | ERR: 57    (40.7%) | Ratio: 1:1.5
VALIDATION SET     | Tot: 20     | OK: 10    (50.0%) | ERR: 10    (50.0%) | Ratio: 1:1.0
TEST SET           | Tot: 40     | OK: 24    (60.0%) | ERR: 16    (40.0%) | Ratio: 1:1.5



In [5]:
# V2: quando accedi a dataset[idx], dove idx è l'indice dello STEP
full_dataset.print_item(0)

STEP DATASET ITEM [0]
Features shape:       torch.Size([39, 1024]) (durata_step, n_features)
Step duration:        39 secondi
Label:                0 (OK)
Step ID:              1
Video ID:             1_10


In [6]:
from utils.inspect_npz import inspect_npz_from_dataset

dataset_folder = DATASET_SOURCE.value
npz_filename = "1_7_360p.mp4_1s_1s.npz"

# Ispezione del file .npz
inspect_npz_from_dataset(full_dataset.features_dir(), npz_filename, n_rows=3)

File: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\data\omnivore\1_7_360p.mp4_1s_1s.npz
Chiavi presenti nel file: ['arr_0']

Array 'arr_0' - shape: (604, 1024), dtype: float32
[[ 0.6910985   0.09298898 -0.6608225  ... -0.75679165  1.2401273
  -0.5683658 ]
 [ 0.40254688 -0.4466254  -0.8645446  ... -1.2709565   0.7917245
  -0.5052321 ]
 [ 0.643613   -0.48683766 -0.88651866 ... -1.0358062   0.658605
  -0.27201462]]


In [7]:
# Inizializzazione W&B
run = wandb.init(
    project="mistake-detection",
    name=f"baseline-LSTM-v3-{DATASET_SOURCE.value}-{SPLIT_TYPE.value}",
    config=config,
    tags=["baseline", "LSTM", DATASET_SOURCE.value],
    notes=f"Baseline LSTM with {DATASET_SOURCE.value} features for mistake detection and {SPLIT_TYPE.value} split"
)

print(f"🚀 W&B Run: {run.name} (ID: {run.id})")

🚀 W&B Run: baseline-LSTM-v3-omnivore-step_id (ID: hruuydk0)


In [8]:
import torch
import torch.nn as nn

model = BaselineV3_LSTM(DATASET_SOURCE.input_dims()).to(device)

# Watch del modello per tracciare gradienti e parametri
wandb.watch(model, log="all", log_freq=10)

In [9]:
lr = config["learning_rate"]
optimizer = torch.optim.Adam(model.parameters(), lr)

# Quanto pesa la classe "positiva" = classe "1" = classe "error":
# - CASO 1: rapporto effettivo del dataset
#train_pos_weight = train_cnt_0 / train_cnt_1

# - CASO 2: rapporto usato nel paper
train_pos_weight = config["pos_weight"]

print(f"Peso classe positiva: {train_pos_weight}")
train_pos_weight = torch.tensor([train_pos_weight], device=device)

criterion = nn.BCEWithLogitsLoss(pos_weight=train_pos_weight)

epochs = config["epochs"]

Peso classe positiva: 1.5


In [10]:
import copy
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score
import numpy as np

best_avg_val_loss = np.inf
final_val_acc = 0
final_val_f1 = 0
final_val_precision = 0
final_val_recall = 0
final_val_auc = 0

for epoch in range(epochs):

    # -------------------------
    #        TRAIN
    # -------------------------
    model.train()
    total_loss = 0
    train_preds_list = []
    train_targets_list = []
    train_probs_list = []

    for inputs, labels, _, _ in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        probs, logits = model(inputs)   # probs: scalare, logits: scalare
        
        loss = criterion(logits, labels)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Metriche train - ogni batch ha una sola predizione (scalare)
        with torch.no_grad():
            pred = (probs >= 0.5).long().item()  # converti a scalare Python
            
            train_preds_list.append(pred)
            train_targets_list.append(labels.item())
            train_probs_list.append(probs.item())

    avg_train_loss = total_loss / len(train_loader)

    # Metriche di training - converti liste a numpy
    train_preds = np.array(train_preds_list)
    train_targets = np.array(train_targets_list)
    train_probs = np.array(train_probs_list)

    train_acc = accuracy_score(train_targets, train_preds)
    train_f1 = f1_score(train_targets, train_preds, zero_division=0)
    train_precision = precision_score(train_targets, train_preds, zero_division=0)
    train_recall = recall_score(train_targets, train_preds, zero_division=0)

    # AUC train (usa probabilità, NON predizioni)
    try:
        train_auc = roc_auc_score(train_targets, train_probs)
    except ValueError:
        train_auc = 0.0  # Caso raro con classe mancante nel batch

    # -------------------------
    #        EVAL
    # -------------------------
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_targets = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels, _, _ in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).float()

            probs, logits = model(inputs)   # probs: scalare, logits: scalare

            val_loss = criterion(logits, labels)
            total_val_loss += val_loss.item()

            # metriche - converti a scalari Python
            pred = (probs >= 0.5).long().item()
            
            all_preds.append(pred)
            all_targets.append(labels.item())
            all_probs.append(probs.item())

        # Converti liste a numpy
        all_preds = np.array(all_preds)
        all_targets = np.array(all_targets)
        all_probs = np.array(all_probs)

        avg_val_loss = total_val_loss / len(val_loader)
        val_acc = accuracy_score(all_targets, all_preds)
        val_f1 = f1_score(all_targets, all_preds, zero_division=0)
        val_precision = precision_score(all_targets, all_preds, zero_division=0)
        val_recall = recall_score(all_targets, all_preds, zero_division=0)

        # AUC validation
        try:
            val_auc = roc_auc_score(all_targets, all_probs)
        except ValueError:
            val_auc = 0.0

        # Confusion Matrix
        cm = confusion_matrix(all_targets, all_preds)

        # Log su W&B
        wandb.log({
            # Training metrics
            "train/loss": avg_train_loss,
            "train/accuracy": train_acc,
            "train/f1": train_f1,
            "train/precision": train_precision,
            "train/recall": train_recall,
            "train/auc": train_auc,

            # Validation metrics
            "val/loss": avg_val_loss,
            "val/accuracy": val_acc,
            "val/f1": val_f1,
            "val/precision": val_precision,
            "val/recall": val_recall,
            "val/auc": val_auc,

            # Confusion Matrix
            "val/confusion_matrix": wandb.plot.confusion_matrix(
                probs=None,
                y_true=all_targets,
                preds=all_preds,
                class_names=["No Error", "Error"]
            ),

            "learning_rate": optimizer.param_groups[0]['lr'],
            "epoch": epoch + 1
        })

        print(f"Epoch {epoch+1}/{epochs} "
            f"- Train Loss: {avg_train_loss:.4f} "
            f"- Val Loss: {avg_val_loss:.4f} "
            f"- Val Acc: {val_acc:.4f} "
            f"- Val F1: {val_f1:.4f} "
            f"- Val Precision: {val_precision:.4f} "
            f"- Val Recall: {val_recall:.4f} "
            f"- Val AUC: {val_auc:.4f}")

        # Salvataggio miglior modello
        if avg_val_loss < best_avg_val_loss:
            best_avg_val_loss = avg_val_loss
            final_val_acc = val_acc
            final_val_f1 = val_f1
            final_val_precision = val_precision
            final_val_recall = val_recall
            final_val_auc = val_auc
            checkpoint_path = os.path.join(ROOT_DIR, "checkpoints", f"best_model_avg_val_loss_{best_avg_val_loss:.4f}.pth")
            os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_f1': val_f1,
                'val_acc': val_acc,
                'val_auc': val_auc,
            }, checkpoint_path)

            artifact = wandb.Artifact(
                name=f"model-{run.id}",
                type="model",
                description=f"Best model with avg_val_loss={best_avg_val_loss:.4f}",
                metadata={
                    "epoch": epoch + 1,
                    "val_f1": val_f1,
                    "val_acc": val_acc,
                    "val_auc": val_auc,
                    "architecture": config["architecture"]
                }
            )
            artifact.add_file(checkpoint_path)
            wandb.log_artifact(artifact)

            print(f"✅ Nuovo miglior modello salvato! avg_val_loss: {best_avg_val_loss:.4f}")

print("\n🎉 Training completato!")
print(f"Miglior avg_val_loss Score: {best_avg_val_loss:.4f}")

Epoch 1/10 - Train Loss: 0.8320 - Val Loss: 0.8608 - Val Acc: 0.6500 - Val F1: 0.5333 - Val Precision: 0.8000 - Val Recall: 0.4000 - Val AUC: 0.7300
✅ Nuovo miglior modello salvato! avg_val_loss: 0.8608
Epoch 2/10 - Train Loss: 0.8166 - Val Loss: 0.8446 - Val Acc: 0.7500 - Val F1: 0.7059 - Val Precision: 0.8571 - Val Recall: 0.6000 - Val AUC: 0.7900
✅ Nuovo miglior modello salvato! avg_val_loss: 0.8446
Epoch 3/10 - Train Loss: 0.7860 - Val Loss: 0.8103 - Val Acc: 0.7000 - Val F1: 0.6667 - Val Precision: 0.7500 - Val Recall: 0.6000 - Val AUC: 0.8200
✅ Nuovo miglior modello salvato! avg_val_loss: 0.8103


KeyboardInterrupt: 

# Test Evaluation

In [ ]:
# -------------------------
#        TEST
# -------------------------
# Carica il miglior modello salvato
checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✅ Modello migliore caricato da: {checkpoint_path}")
print(f"   Epoch: {checkpoint['epoch']}, Val F1: {checkpoint['val_f1']:.4f}, Val Acc: {checkpoint['val_acc']:.4f}")

model.eval()
total_test_loss = 0
test_preds_list = []
test_targets_list = []
test_probs_list = []

with torch.no_grad():
    for inputs, labels, _, _ in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        probs, logits = model(inputs)   # probs: scalare, logits: scalare

        test_loss = criterion(logits, labels)
        total_test_loss += test_loss.item()

        # metriche - converti a scalari Python
        pred = (probs >= 0.5).long().item()
        
        test_preds_list.append(pred)
        test_targets_list.append(labels.item())
        test_probs_list.append(probs.item())

    # Converti liste a numpy
    test_preds = np.array(test_preds_list)
    test_targets = np.array(test_targets_list)
    test_probs = np.array(test_probs_list)

    avg_test_loss = total_test_loss / len(test_loader)
    test_acc = accuracy_score(test_targets, test_preds)
    test_f1 = f1_score(test_targets, test_preds, zero_division=0)
    test_precision = precision_score(test_targets, test_preds, zero_division=0)
    test_recall = recall_score(test_targets, test_preds, zero_division=0)

    # AUC test
    try:
        test_auc = roc_auc_score(test_targets, test_probs)
    except ValueError:
        test_auc = 0.0

    # Confusion Matrix
    cm_test = confusion_matrix(test_targets, test_preds)

    # Log su W&B
    wandb.log({
        "test/loss": avg_test_loss,
        "test/accuracy": test_acc,
        "test/f1": test_f1,
        "test/precision": test_precision,
        "test/recall": test_recall,
        "test/auc": test_auc,

        # Confusion Matrix
        "test/confusion_matrix": wandb.plot.confusion_matrix(
            probs=None,
            y_true=test_targets,
            preds=test_preds,
            class_names=["No Error", "Error"]
        )
    })

    print(f"\n📊 Test Results:")
    print(f"Test Loss: {avg_test_loss:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test F1: {test_f1:.4f}")
    print(f"Test Precision: {test_precision:.4f}")
    print(f"Test Recall: {test_recall:.4f}")
    print(f"Test AUC: {test_auc:.4f}")
    print(f"\n📈 Confusion Matrix:")
    print(cm_test)

✅ Modello migliore caricato da: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\checkpoints\best_model_avg_val_loss_0.8103.pth
   Epoch: 3, Val F1: 0.6667, Val Acc: 0.7000

📊 Test Results:
Test Loss: 0.7823
Test Accuracy: 0.6750
Test F1: 0.5517
Test Precision: 0.6154
Test Recall: 0.5000
Test AUC: 0.7448

📈 Confusion Matrix:
[[19  5]
 [ 8  8]]


wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.
